<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 8
## <center> Vowpal Wabbit в задаче прогнозирования популярности статьи на хабре

В этом задании надо побить бенчмарк в [соревновании](https://www.kaggle.com/c/habr-num-bookmarks) на Kaggle Inclass. Как это делать – ограничений нет (кроме, конечно, ручной разметки), прочитать правила можно [тут](https://www.kaggle.com/c/habr-num-bookmarks/rules). Ниже описаны инструкции, как это сделать с Vowpal Wabbit.

Дедлайн: 31 октября 23:59 UTC +3. Решение надо будет загрузить по [ссылке](https://www.dropbox.com/request/g5WOPrxwvcYwADZCuoY7). В этом соревновании нет задачи победить. Цель – побить бенчмарк и продвинуться в [соревновании](https://mlcourse.arktur.io) по прогнозу популярности статьи на Medium. 

In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [2]:
!head -1 ../../data/hw8_vw/train.json > ../../data/hw8_vw/train1.json

In [4]:
with open('../../data/hw8_vw/train1.json') as inp_json:
    first_json = json.load(inp_json)

In [5]:
first_json.keys()

dict_keys(['author', 'content', 'hubs', 'published', 'flags', 'post_id', '_id', 'polling', 'tags', 'title', 'flow', '_timestamp', 'meta_tags', 'url', 'domain', 'link_tags'])

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

In [6]:
first_json['_id']

'https://habrahabr.ru/post/7/'

In [7]:
first_json['_timestamp']

1493192186.0903192

In [29]:
from datetime import datetime
dt = datetime.fromtimestamp(first_json['_timestamp'])
dt.month, dt.hour

(4, 7)

In [8]:
first_json['url']

'https://habrahabr.ru/post/7/'

In [9]:
first_json['domain']

'habrahabr.ru'

In [27]:
first_json['published']['$date']

'2006-07-15T01:48:00.000Z'

In [11]:
first_json['title']

'Самопроизвольное разлогинивание'

In [12]:
first_json['content']

'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.<br>\r\n<br>\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.'

In [13]:
first_json['polling']

In [14]:
first_json['post_id']

7

In [15]:
first_json['flags']

[]

In [16]:
first_json['hubs']

[{'id': 'hub/habr',
  'title': 'Хабрахабр',
  'url': 'https://habrahabr.ru/hub/habr/'}]

In [17]:
first_json['flow']

In [18]:
first_json['tags']

['логин', 'login']

In [18]:
first_json['author']

{'name': 'Павел Титов',
 'nickname': '@ptitov',
 'url': 'https://habrahabr.ru/users/ptitov'}

In [19]:
first_json['link_tags']

{'alternate': 'https://habrahabr.ru/rss/post/7/',
 'apple-touch-icon-precomposed': '/images/favicons/apple-touch-icon-152x152.png',
 'canonical': 'https://habrahabr.ru/post/7/',
 'icon': '/images/favicons/favicon-16x16.png',
 'image_src': 'https://habrahabr.ru/i/habralogo.jpg',
 'stylesheet': 'https://habracdn.net/habr/styles/1493134745/_build/global_main.css'}

In [20]:
first_json['meta_tags']

{'al:android:app_name': 'Habrahabr',
 'al:android:package': 'ru.habrahabr',
 'al:android:url': 'habrahabr://post/7',
 'al:windows_phone:app_id': '460a6bd6-8955-470f-935e-9ea1726a6060',
 'al:windows_phone:app_name': 'Habrahabr',
 'al:windows_phone:url': 'habrahabr://post/7',
 'apple-mobile-web-app-title': 'Хабрахабр',
 'application-name': 'Хабрахабр',
 'description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.',
 'fb:app_id': '444736788986613',
 'keywords': 'логин, login',
 'msapplication-TileColor': '#FFFFFF',
 'msapplication-TileImage': 'mstile-144x144.png',
 'og:description': 'У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.  Возможно, это происходит при...',
 'og:image': 'https://habrahabr.ru/i/habralogo.jpg',
 'og:title': 'Самопроизвольное разлогинивание'

Загрузим ответы на обучающей выборке.

In [21]:
train_target = pd.read_csv('../../data/hw8_vw/train_target.csv',
                          index_col='url')

In [22]:
train_target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120000 entries, https://habrahabr.ru/post/7/ to https://geektimes.ru/post/87438/
Data columns (total 1 columns):
target    120000 non-null float64
dtypes: float64(1)
memory usage: 1.8+ MB


In [23]:
train_target.head()

,target
url,
https://habrahabr.ru/post/7/,0.693147
https://geektimes.ru/post/11/,1.098612
https://geektimes.ru/post/112/,0.000000
https://geektimes.ru/post/1127/,0.000000
https://geektimes.ru/post/12664/,0.000000


Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, title, tags, domain, flow, author, и hubs из JSON-файла.
От самого текста для начала просто возьмем его длину – постройте признак content_len – длина текста в миллионнах символов.
Также постройте признаки – час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`. Ниже пример того, как могут выглядеть первые две строки нового файла.

In [140]:
!head -2 ../../data/hw8_vw/habr_train.vw

0.693147 |title b'\\u0421\\u0430\\u043c\\u043e\\u043f\\u0440\\u043e\\u0438\\u0437\\u0432\\u043e\\u043b\\u044c\\u043d\\u043e\\u0435 \\u0440\\u0430\\u0437\\u043b\\u043e\\u0433\\u0438\\u043d\\u0438\\u0432\\u0430\\u043d\\u0438\\u0435' |num content_len:199.0 month:7 hour:1
1.098612 |title b'Stand-along c\\u043e\\u043e\\u0431\\u0449\\u0435\\u0441\\u0442\\u0432\\u0430 \\u043f\\u0440\\u043e\\u0442\\u0438\\u0432 \\u0441\\u043e\\u043e\\u0431\\u0449\\u0435\\u0441\\u0442\\u0432 \\u0432 \\u0440\\u0430\\u043c\\u043a\\u0430\\u0445 \\u0441\\u043e\\u0446\\u0438\\u0430\\u043b\\u044c\\u043d\\u044b\\u0445 \\u0441\\u0435\\u0442\\u0435\\u0439' |num content_len:988.0 month:7 hour:14


In [141]:
%%time
from datetime import datetime

with open('../../data/hw8_vw/train.json') as inp_json, \
     open('../../data/hw8_vw/habr_train.vw', 'w') as out_vw:
    n = 0
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        
        # Ваш код здесь
        out = []
        
        out.append(train_target.loc[data_json["url"]].to_string(index=False) + " ")
        out.append("title {} ".format(data_json["title"].encode("unicode")))
#         out.append("tags {} ".format(" ".join(data_json["tags"])))
#         out.append("domain {} ".format(data_json["domain"]))
#         out.append("flow {} ".format(data_json["flow"] if data_json["flow"] else "None"))
#         out.append("author {} ".format(data_json["author"]["nickname"]))
#         out.append("hubs {} ".format(" ".join([d["title"] for d in data_json["hubs"]])))
        dt = datetime.strptime(data_json["published"]["$date"][:-1], "%Y-%m-%dT%H:%M:%S.%f")       
        out.append("num content_len:{} month:{} hour:{}\n".format(
            float(len(data_json["content"])), 
            dt.month, 
            dt.hour))
        out = "|".join(out)
        
        out_vw.write(out)
        n += 1
        if n == 2: break

LookupError: unknown encoding: unicode

Проделайте все то же с тестовой выборкой, вместо ответов подсовывая что угодно, например, единицы.

In [112]:
with open('../../data/hw8_vw/test.json') as inp_json, \
     open('../../data/hw8_vw/habr_test.vw', 'w', encoding="utf8") as out_vw:
    for line in tqdm_notebook(inp_json):
        data_json = json.loads(line)
        
        # Ваш код здесь
        out = []
        
        out.append("1 ")
        out.append("title {} ".format(data_json["title"]))
        out.append("tags {} ".format(" ".join(data_json["tags"])))
        out.append("domain {} ".format(data_json["domain"]))
        out.append("flow {} ".format(data_json["flow"] if data_json["flow"] else "None"))
        out.append("author {} ".format(data_json["author"]["nickname"]))
        out.append("hubs {} ".format(" ".join([d["title"] for d in data_json["hubs"]])))
        dt = datetime.strptime(data_json["published"]["$date"][:-1], "%Y-%m-%dT%H:%M:%S.%f")       
        out.append("num content_len:{} month:{} hour:{}\n".format(
            float(len(data_json["content"])), 
            dt.month, 
            dt.hour))
        out = "|".join(out)
        out_vw.write(out)        

In [122]:
!head -2 ../../data/hw8_vw/habr_test.vw

1 |title �������� ����! |tags ���� Pi |domain geektimes.ru |flow None |author @Timursan |hubs ������������ �������� |num content_len:228.0 month:3 hour:3
1 |title ������������ ������ ������������������ �������������� ���������������������� CD ���� ���������� �� ���������������������� �� ������������ FLAC |tags bash lossless |domain geektimes.ru |flow None |author @da3mon |hubs ������������ �������� |num content_len:9377.0 month:3 hour:0


Выбор того, как валидировать модель, остается за Вами. Проще всего, конечно, сделать отложенную выборку. Бенчмарк, который Вы видите в соревновании (**vw_baseline.csv**) и который надо побить, получен с Vowpal Wabbit, 3 проходами по выборке (не забываем удалять кэш), биграммами и настроенными гиперпараметрами `bits`, `learning_rate` и `power_t`. 

In [ ]:
# Ваш код здесь

In [25]:
sample_sub = pd.read_csv('../../data/sample_submission.csv', index_col='url')

In [26]:
sample_sub.head()

,target
url,
https://geektimes.ru/post/87455/,11.620054
https://geektimes.ru/post/87452/,4.822528
https://geektimes.ru/post/87459/,0.921104
https://habrahabr.ru/post/87461/,1.632126
https://habrahabr.ru/post/5754/,1.952122


In [ ]:
your_submission = sample_sub.copy()
your_submission['target'] = # Ваш код здесь
your_submission.to_csv('submission.csv')